# Asistente Virtual para Consultas Médicas

## 1. Introducción

Este notebook describe la implementación de un asistente virtual especializado en consultas médicas, capaz de procesar, indexar y responder preguntas basadas en documentos médicos en formato PDF.

El programa está compuesto por tres módulos principales:

1. **Generador de Consultas** (`generator.py`): Crea documentos PDF simulando consultas médicas reales.
2. **Indexador de Documentos** (`indexer.py`): Procesa los PDFs y genera embeddings para búsqueda semántica.
3. **Agente Conversacional** (`agent.py`): Implementa el asistente virtual que interactúa con el usuario.

El código completo se encuentra disponible en [github.com/MartinCastroAlvarez/langchain-virtual-assistant](https://github.com/MartinCastroAlvarez/langchain-virtual-assistant)

## 2. Instalación de Dependencias

In [1]:
!pip install langchain langchain-openai sentence-transformers colorama fpdf2 pymupdf beautifulsoup4 numpy scikit-learn tqdm


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## 3. Carga de Datos desde Documentos Externos

El programa utiliza `generator.py` para crear documentos PDF que simulan consultas médicas reales. Este módulo incluye:

In [2]:
from generator import Consultation

for _ in range(1000):
    consulta = Consultation.generate()
    filename = consulta.to_pdf()
    print(f"Consulta generada en: {filename}")

/Users/martincastro/Library/Caches/pypoetry/virtualenvs/langchain-virtual-assistant-aF0DWEuW-py3.10/lib/python3.10/site-packages/fpdf/__init__.py:40: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


PDF generado: pdfs/David_Paredes_30-10-2024.pdf
Consulta generada en: pdfs/David_Paredes_30-10-2024.pdf
PDF generado: pdfs/Pedro_Ramírez_26-04-2024.pdf
Consulta generada en: pdfs/Pedro_Ramírez_26-04-2024.pdf
PDF generado: pdfs/Sofía_Castro_07-04-2025.pdf
Consulta generada en: pdfs/Sofía_Castro_07-04-2025.pdf
PDF generado: pdfs/Agustina_Torres_04-12-2024.pdf
Consulta generada en: pdfs/Agustina_Torres_04-12-2024.pdf
PDF generado: pdfs/Liam_Navarro_14-07-2024.pdf
Consulta generada en: pdfs/Liam_Navarro_14-07-2024.pdf
PDF generado: pdfs/María_González_09-09-2024.pdf
Consulta generada en: pdfs/María_González_09-09-2024.pdf
PDF generado: pdfs/Oliver_Castro_27-05-2024.pdf
Consulta generada en: pdfs/Oliver_Castro_27-05-2024.pdf
PDF generado: pdfs/Santiago_León_11-01-2025.pdf
Consulta generada en: pdfs/Santiago_León_11-01-2025.pdf
PDF generado: pdfs/Andrea_Mendoza_23-05-2024.pdf
Consulta generada en: pdfs/Andrea_Mendoza_23-05-2024.pdf
PDF generado: pdfs/David_Paredes_09-06-2024.pdf
Consulta gen

El programa utiliza `PyPDFLoader` de LangChain para cargar los documentos PDF dentro de `indexer.py` que se encarga de generar una base de datos de embeddings en el archivo `vectorstore.json`

In [3]:
from indexer import Indexer

Indexer().run()

Document 1/3972
Loading embedding model: all-MiniLM-L6-v2...
Model loaded.
Document 2/3972
Document 3/3972
Document 4/3972
Document 5/3972
Document 6/3972
Document 7/3972
Document 8/3972
Document 9/3972
Document 10/3972
Document 11/3972
Document 12/3972
Document 13/3972
Document 14/3972
Document 15/3972
Document 16/3972
Document 17/3972
Document 18/3972
Document 19/3972
Document 20/3972
Document 21/3972
Document 22/3972
Document 23/3972
Document 24/3972
Document 25/3972
Document 26/3972
Document 27/3972
Document 28/3972
Document 29/3972
Document 30/3972
Document 31/3972
Document 32/3972
Document 33/3972
Document 34/3972
Document 35/3972
Document 36/3972
Document 37/3972
Document 38/3972
Document 39/3972
Document 40/3972
Document 41/3972
Document 42/3972
Document 43/3972
Document 44/3972
Document 45/3972
Document 46/3972
Document 47/3972
Document 48/3972
Document 49/3972
Document 50/3972
Document 51/3972
Document 52/3972
Document 53/3972
Document 54/3972
Document 55/3972
Document 56/397

KeyboardInterrupt: 

El programa `agent.py` define una clase `PDFVectorRetriever` para poder encontrar los documents PDF que mejor se relacionan con la pregunta del usuario.

In [5]:
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document as LC_Document

class PDFVectorRetriever(BaseRetriever):
    def get_relevant_documents(self, query: str) -> list[LC_Document]:
        query_embedding = Vector.model.encode(query)
        top_docs = Store.search(query_embedding, n=3)
        docs = []
        for doc, _ in top_docs:
            filepath = os.path.join(PDF_DIR, doc.filename)
            loader = PyPDFLoader(filepath)
            pages = loader.load()
            for page in pages:
                docs.append(LC_Document(page_content=page.page_content, metadata={"source": doc.filename}))
        return docs

    async def aget_relevant_documents(self, query: str) -> list[LC_Document]:
        return self.get_relevant_documents(query)

/var/folders/92/tj16j9915sb7pmdckykvjx980000gn/T/ipykernel_3073/3671330362.py:4: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class PDFVectorRetriever(BaseRetriever):
/var/folders/92/tj16j9915sb7pmdckykvjx980000gn/T/ipykernel_3073/3671330362.py:4: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class PDFVectorRetriever(BaseRetriever):


## 4. Procesamiento de Documentos

El programa utiliza `RecursiveCharacterTextSplitter` en `indexer.py` para indexar los PDFs de forma parcial, para evitar el problema de archivos muy pesados.

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Configuración del splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
)

Además, en `indexer.py` se define una clase `Pdf` que se encarga de limpiar los PDFs de términos comunes a todos los documentos tales como "fecha", "paciente", "doctor". De ese modo, se indexan sólamente los términos más importantes tales como "dolor de cabeza", "ansiedad", etc.

In [7]:
import os
import re
from dataclasses import dataclass
from indexer import BOILERPLATE_PATTERNS, COMMON_WORDS, TEXT_SPLITTER

@dataclass
class Pdf:
    filepath: str

    @property
    def filename(self) -> str:
        return os.path.basename(self.filepath)

    def clean(self, text: str) -> str:
        cleaned = text.lower()
        for pattern in BOILERPLATE_PATTERNS:
            cleaned = re.sub(pattern, "", cleaned, flags=re.IGNORECASE)
        words = cleaned.split()
        return " ".join(word for word in words if word.lower() not in COMMON_WORDS)

    def split(self) -> list[str]:
        doc = fitz.open(self.filepath)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        text = self.clean(" ".join(text.split()))
        chunks = TEXT_SPLITTER.split_text(text)
        return [chunk for chunk in chunks if len(chunk.split()) > 5]

## 5. Embeddings

El programa utiliza el modelo `all-MiniLM-L6-v2` de Sentence Transformers para generar embeddings.

In [8]:
from sentence_transformers import SentenceTransformer
from dataclasses import dataclass, field
from indexer import PDF_DIR, DATABASE_FILE, EMBEDDING_MODEL, CACHE_DIR

@dataclass
class Indexer:
    pdf_dir: str = field(default=PDF_DIR)
    db_file: str = field(default=DATABASE_FILE)
    model_name: str = field(default=EMBEDDING_MODEL)
    cache_dir: str = field(default=CACHE_DIR)
    _model: SentenceTransformer | None = None

    @property
    def model(self) -> SentenceTransformer:
        if self._model is None:
            print(f"Loading embedding model: {self.model_name}...")
            self._model = SentenceTransformer(self.model_name, cache_folder=self.cache_dir)
            print("Model loaded.")
        return self._model

    def run(self):
        assert os.path.exists(self.pdf_dir), f"Error: Directory '{self.pdf_dir}' not found."
        pdf_filepaths = glob.glob(os.path.join(self.pdf_dir, "*.pdf"))
        assert pdf_filepaths, f"No PDF files found in '{self.pdf_dir}'."
        documents: list[Document] = []
        for i, filepath in enumerate(pdf_filepaths, 1):
            print(f"Document {i}/{len(pdf_filepaths)}")
            pdf = Pdf(filepath)
            document = Document(filename=pdf.filename, text="", embeddings=[])
            chunks = pdf.split()
            if chunks:
                document.text = chunks[0]
                chunk_embeddings = self.model.encode(chunks, convert_to_numpy=True)
                document.embeddings = [embedding.tolist() for embedding in chunk_embeddings]
                documents.append(document)

        print(f"Processed {len(documents)} documents")
        print("Writing database...")
        database = [doc.to_dict() for doc in documents]
        with open(self.db_file, "w") as f:
            json.dump(database, f, indent=4)

        print(f"Database successfully created at {self.db_file}")

Por otra parte, el programa `agent.py` también implementa algo similar en `PDFVectorRetriever` pero con la consulta del usuario. De esa forma, es posible calcular de forma numérica la distancia entre los embeddings de documentos diferentes.

In [10]:
import numpy as np

class Vector:
    model: SentenceTransformer | None = None

    @classmethod
    def load(cls) -> None:
        cls.model = SentenceTransformer(EMBEDDING_MODEL_NAME, cache_folder=CACHE_DIR)
        Out.green(f"Vector model loaded with {len(cls.model.encode('test'))} dimensions")

    @classmethod
    def distance(cls, query_embedding: np.ndarray, doc_embedding: np.ndarray) -> float:
        query_embedding = query_embedding.flatten()
        doc_embedding = doc_embedding.flatten()
        query_embedding = query_embedding.reshape(1, -1)
        doc_embedding = doc_embedding.reshape(1, -1)
        return float(cosine_similarity(query_embedding, doc_embedding)[0][0])

## 6. Modelo de Lenguaje (LLM)

El programa utiliza el modelo `GPT-3.5-turbo` de OpenAI. La configuración se realiza en la clase `Brain` y requiere la variable de entorno `OPENAI_API_KEY`:

In [11]:
from langchain_openai import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = "reemplazar-aqui"
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

## 7. PromptTemplate

El programa utiliza dos templates principales definidos en la clase `Template`:

In [12]:
from langchain.prompts import PromptTemplate

class Template:
    RAG = PromptTemplate(
        input_variables=["input", "context"],
        template=(
            "You are an assistant specializing in analyzing medical consultation PDFs. "
            "Answer the following question based *only* on the provided context from relevant PDF documents. "
            "If the context doesn't contain the answer, state that the information is not available in the provided documents. "
            "Explicitly mention the filename(s) from the context that support your answer. The context contains markers like '--- Context from filename.pdf ---'.\n\n"
            "Context from PDF documents:\n{context}\n\n"
            "Question from the patient: {input}\n"
            "Your Answer:"
        ),
    )

    TRANSLATE = PromptTemplate(
        input_variables=["text"],
        template=(
            "Translate the following English medical text to Spanish, using simple and clear language that a non-medical audience can understand. "
            "If there are medical terms, provide simple explanations in parentheses. Keep the tone friendly and accessible.\n\n"
            "English text: {text}\n\n"
            "Simple Spanish translation:"
        ),
    )

## 8. Memoria

El programa mantiene un resúmen de la conversación en memoria en `agent.py`:

In [17]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.agents import initialize_agent
from langchain.chains.combine_documents import create_stuff_documents_chain
from agent import Brain, PDFVectorRetriever, Conversation
from langchain.agents import AgentExecutor

Brain.load()

memory: ConversationSummaryBufferMemory = Conversation.load(Brain.model)
retriever = PDFVectorRetriever()
combine_docs_chain = create_stuff_documents_chain(llm=Brain.model, prompt=Template.RAG)
rag_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=combine_docs_chain,
)
executor: AgentExecutor = initialize_agent(
    tools=[],
    llm=Brain.model,
    agent="chat-conversational-react-description",
    verbose=True,
    memory=memory,
    handle_parsing_errors=True,
    agent_kwargs={
        "system_message": (
            "You are a bilingual (English-Spanish) medical assistant specializing in analyzing medical consultation PDFs. "
            "Always respond in the same language as the user's question. "
            "For English questions, answer in English. For Spanish questions, answer in Spanish. "
            "When a user presents any medical symptoms or health-related questions, ALWAYS use the Recommend tool first "
            "to search through the medical PDFs and provide evidence-based information. "
            "When answering in Spanish, use simple and clear language that a non-medical audience can understand, "
            "and include brief explanations in parentheses for medical terms. "
            "Base your answers on the provided context from relevant PDF documents and always reference which documents "
            "you used in your response. If the medical information needed is not found in the PDFs, clearly state this "
            "and suggest consulting a healthcare professional."
        )
    },
)

Brain model loaded with gpt-3.5-turbo
Loading conversation history from /tmp/conversation_cache.pkl


## 9. Ejemplo

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from agent import Agent

Agent.load()
agent = Agent()

Brain model loaded with gpt-3.5-turbo
Store loaded with 1383 documents and 1383 total embeddings
Vector model loaded with 384 dimensions
Loading conversation history from /tmp/conversation_cache.pkl


/Users/martincastro/viu/langchain-virtual-assistant/agent.py:186: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=max_token_limit, memory_key="chat_history", return_messages=True)
/Users/martincastro/viu/langchain-virtual-assistant/agent.py:239: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/h

In [3]:
agent.ask("tengo dolor de cabeza")



> Entering new AgentExecutor chain...


/Users/martincastro/viu/langchain-virtual-assistant/agent.py:276: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.executor.run(input=query)


```json
{
    "action": "Recommend",
    "action_input": "dolor de cabeza"
}
```Top 3 similarities: [0.61195749026703, 0.5695069567326809, 0.5421169904309269]
Top 3 documents: ['Catalina_Ríos_02-05-2024.pdf', 'Isabella_Romero_28-06-2024.pdf', 'Oliver_Castro_13-12-2024.pdf']

Observation: The patient with the complaint of "dolor de cabeza" (headache) was Catalina Ríos. This information is available in the "Informe de Consulta Médica" for Catalina Ríos, dated 02/05/2024, with Dr. Sergio Romero as the attending physician.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "La paciente con la queja de dolor de cabeza fue Catalina Ríos. Esta información está disponible en el 'Informe de Consulta Médica' para Catalina Ríos, con fecha 02/05/2024, y el médico que la atendió fue el Dr. Sergio Romero."
}
```

> Finished chain.


"La paciente con la queja de dolor de cabeza fue Catalina Ríos. Esta información está disponible en el 'Informe de Consulta Médica' para Catalina Ríos, con fecha 02/05/2024, y el médico que la atendió fue el Dr. Sergio Romero."

In [4]:
agent.ask("tengo ansiedad y depresión, me podría ayudar por favor?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Recommend",
    "action_input": "ansiedad depresión"
}
```Top 3 similarities: [0.4609630662327597, 0.45725124908671616, 0.4386712385624667]
Top 3 documents: ['Lucas_Herrera_07-09-2024.pdf', 'Alejandro_Martínez_30-09-2024.pdf', 'William_Ríos_20-01-2025.pdf']

Observation: The information regarding anxiety is not available in the provided documents. The only information available is related to the diagnosis and treatment of depression in the patients Lucas Herrera, Alejandro Martínez, and William Ríos. (Filename: Informe_Consulta_Medica_Lucas_Herrera.pdf, Informe_Consulta_Medica_Alejandro_Martinez.pdf, Informe_Consulta_Medica_William_Rios.pdf)
Thought:```json
{
    "action": "Final Answer",
    "action_input": "La información sobre la ansiedad no está disponible en los documentos proporcionados. Sin embargo, la información disponible se relaciona con el diagnóstico y tratamiento de la depresión en los pacientes Lucas Herrer

'La información sobre la ansiedad no está disponible en los documentos proporcionados. Sin embargo, la información disponible se relaciona con el diagnóstico y tratamiento de la depresión en los pacientes Lucas Herrera, Alejandro Martínez y William Ríos.'